In [1]:
import cv2 as cv

import glob
import numpy as np

In [2]:
%matplotlib tk
import matplotlib.pyplot as plt
import numpy as np


from PIL import Image

In [3]:
left_image_path = 'data/7_left.JPG'
right_image_path = 'data/7_right.JPG'


im_left = cv.imread(left_image_path, 1)

im_right = cv.imread(right_image_path, 1)
# plt.figure()
# plt.imshow(im_left) 
# plt.show()

In [4]:
# idea here we need a chessboard to calibrate the camera, world_scaling can be the real size 
rows = 6
cols = 7
world_scaling = 1

In [5]:
def find_carmera_matrix(image_path):
    image = cv.imread(image_path)
    objp = np.zeros((rows*cols,3), np.float32)
    objp[:,:2] = np.mgrid[0:rows,0:cols].T.reshape(-1,2)
    objp = objp * world_scaling
    width = image.shape[1]
    height = image.shape[0]   

    img_point = []
    points_3d = []
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    ret, corners = cv.findChessboardCorners(gray, (rows,cols), cv.CALIB_CB_NORMALIZE_IMAGE)
    if ret == True:
        corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        img_point.append(corners2)
        points_3d.append(objp)
        cv.drawChessboardCorners(image, (rows,cols), corners2, ret)
        cv.imshow('image', image)
        cv.waitKey(0)
        cv.destroyAllWindows()
    else:
        print('corners not found')
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(points_3d,img_point, (width, height), None, None)
    print(ret)
    return mtx, dist, img_point, points_3d



In [6]:
mtx_R, dist_R, img_point_R, points_3d_R = find_carmera_matrix(right_image_path)

mtx_L, dist_L, img_point_L, points_3d_L = find_carmera_matrix(left_image_path)


 
# print(points_3d_R)
# print(points_3d_L)0

2024-12-17 19:46:33.976 python[16769:8260304] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-17 19:46:33.976 python[16769:8260304] +[IMKInputSession subclass]: chose IMKInputSession_Modern


0.15480580288616166
0.10368479668552631


In [7]:
print(dist_L)
print(dist_R)


width = im_left.shape[1]
height = im_left.shape[0]
print(width, height)
width = im_right.shape[1]
height = im_right.shape[0]
print(width, height)

[[ 4.67477052e-01 -3.66964417e+00  6.38064550e-03 -1.78085008e-02
   1.03751327e+01]]
[[ 0.10413267 -0.56309796  0.02129309  0.00703079 -4.49056238]]
5712 4284
5712 4284


In [8]:
def find_stereo_camera_matrix(mtx_L, dist_L, mtx_R, dist_R, img_point_L, img_point_R, points_3d):
    stereocalibration_flags = cv.CALIB_FIX_INTRINSIC
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.0001)
    ret, CM1, dist1, CM2, dist2, R, T, E, F = cv.stereoCalibrate(points_3d, img_point_L, img_point_R, mtx_L, dist_L,
                                mtx_R, dist_R, (width, height), criteria = criteria, flags = stereocalibration_flags)
    return R, T


In [9]:
R,T = find_stereo_camera_matrix(mtx_L, dist_L, mtx_R, dist_R, img_point_L, img_point_R, points_3d_L)

In [10]:
RT1 = np.concatenate([np.eye(3), [[0],[0],[0]]], axis = -1)
P1 = mtx_L @ RT1

RT2 = np.concatenate([R, T], axis = -1)
P2 = mtx_R @ RT2


In [11]:
def find_center(M, shown = False):
    U, S, V = np.linalg.svd(M)
    if shown:
        print(S)
        print("Camera Center:", V[-1, :3] / V[-1, -1])
    return V[-1, :3] / V[-1, -1]

In [12]:
def DLT(P1, P2, point1, point2):
 
    A = [point1[1]*P1[2,:] - P1[1,:],
         P1[0,:] - point1[0]*P1[2,:],
         point2[1]*P2[2,:] - P2[1,:],
         P2[0,:] - point2[0]*P2[2,:]
        ]
    A = np.array(A).reshape((4,4))
    #print('A: ')
    #print(A)
 
    B = A.transpose() @ A
    from scipy import linalg
    U, s, Vh = linalg.svd(B, full_matrices = False)
 
    # print('Triangulated point: ')
    # print(Vh[3,0:3]/Vh[3,3])
    return Vh[3,0:3]/Vh[3,3]

In [13]:
pt = DLT(P1, P2, img_point_L[0][0][0], img_point_R[0][0][0])
print(pt)

[20.38241371  3.32959346 69.38126954]


In [14]:
pt = DLT(P1, P2, img_point_L[0][1][0], img_point_R[0][1][0])

In [15]:
c1 = find_center(P1)
c2 = find_center(P2)

print(c1)
print(c2)


[0. 0. 0.]
[35.63069593 -7.357921   23.75061013]


In [16]:
def get_input_lines(im, min_lines=3):
    n = 0

    plt.figure()
    plt.imshow(im)
    plt.show()
    print('Set 3 points to give the plane of table')

    clicked = plt.ginput(3, timeout=0, show_clicks=True)
    if not clicked or len(clicked) < 3:
        print('Need at least %d lines, you have %d now' % (min_lines, n))


    
    pt1 = np.array([clicked[0][0], clicked[0][1], 1])
    pt2 = np.array([clicked[1][0], clicked[1][1], 1])
    pt3 = np.array([clicked[2][0], clicked[2][1], 1])

    return [pt1, pt2, pt3]

In [17]:
pts_left = (get_input_lines(im_left, 3))
pts_right = (get_input_lines(im_right, 3))

Set 3 points to give the plane of table
Set 3 points to give the plane of table


In [37]:
pts_x = []
pts_y = []
pts_z = []

for i in range(img_point_L[0].shape[0]):
    pt = DLT(P1, P2, img_point_L[0][i][0], img_point_R[0][i][0])
    # pt = pt / pt[2]
    pts_x.append(pt[0])
    pts_y.append(pt[1])
    pts_z.append(pt[2])
print(len(pts_x), len(pts_y), len(pts_z))


42 42 42


In [19]:
pt_table = np.array([DLT(P1, P2, pts_left[0], pts_right[0]), DLT(P1, P2, pts_left[1], pts_right[1]), DLT(P1, P2, pts_left[2], pts_right[2])])

v1 = pt_table[0] - pt_table[1]
v2 = pt_table[0] - pt_table[2]
normal = np.cross(v1, v2)

d = -np.dot(normal, pt_table[0])



In [20]:
fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')

pts_x_c = pts_x + [c1[0], c2[0]]
pts_y_c = pts_y + [c1[1], c2[1]]
pts_z_c = pts_z + [c1[2], c2[2]]


max_range = max([max(pts_x_c)-min(pts_x_c), max(pts_y_c)-min(pts_y_c), max(pts_z_c)-min(pts_z_c)])
mid_x = (max(pts_x_c) + min(pts_x_c)) * 0.5
mid_y = (max(pts_y_c) + min(pts_y_c)) * 0.5
mid_z = (max(pts_z_c) + min(pts_z_c)) * 0.5
ax.view_init(elev=135, azim=0)
ax.set_xlim(mid_x - max_range/2, mid_x + max_range/2)
ax.set_ylim(mid_y - max_range/2, mid_y + max_range/2)
ax.set_zlim(mid_z - max_range/2, mid_z + max_range/2)

ax.scatter(pts_x, pts_y, pts_z, c='b')  
ax.scatter(c1[0], c1[1], c1[2], c='r')
ax.scatter(c2[0], c2[1], c2[2], c='r')
ax.scatter(pt_table[0][0], pt_table[0][1], pt_table[0][2], c='g')
ax.scatter(pt_table[1][0], pt_table[1][1], pt_table[1][2], c='g')
ax.scatter(pt_table[2][0], pt_table[2][1], pt_table[2][2], c='g')
X_range = np.linspace(mid_x - max_range/2, mid_x + max_range/2, 10)
Y_range = np.linspace(mid_y - max_range/2, mid_y + max_range/2, 10)
ax.legend(['Chessboard', 'Camera 1', 'Camera 2', 'Table'])
n_x, n_y, n_z = normal
X, Y = np.meshgrid(X_range, Y_range)
Z = (-d - n_x * X - n_y * Y) / n_z
ax.plot_surface(X, Y, Z, alpha=0.5, color='cyan')
plt.show()

In [21]:

def find_plane_two_vectors(c, d1, d2):
    """
    Find the plane passing through the camera center and spanned by two direction vectors.

    Parameters:
        c (numpy.ndarray): Camera center coordinates (3D point).
        d1 (numpy.ndarray): First direction vector (3D).
        d2 (numpy.ndarray): Second direction vector (3D).

    Returns:
        n (numpy.ndarray): Normalized normal vector of the plane.
        d (float): Plane constant such that n . x = d for any point x on the plane.
    """
    # Step 1: Compute the normal vector to the plane
    n = np.cross(d1, d2)  # Cross product gives the perpendicular vector
    if np.linalg.norm(n) == 0:
        raise ValueError("Direction vectors are collinear; no unique plane can be defined.")
    
    n = n / np.linalg.norm(n)  # Normalize the normal vector
    # Step 2: Compute the plane constant 'd' using the camera center 'c'
    d = -np.dot(n, c)  # Plane equation: n . x + d = 0

    return n, d

In [22]:

def backproject(P, x):
    """
    Backproject a 2D image point x into a 3D ray given a camera projection matrix P.

    Parameters:
        P (numpy.ndarray): 3x4 camera projection matrix.
        x (numpy.ndarray): 2D image point in homogeneous coordinates [u, v, 1].
    
    Returns:
        d (numpy.ndarray): 3D ray direction vector.
    """

    # Step 2: Construct the backprojected ray direction
    # Solve for a point along the ray: X = [X, Y, Z, 1] such that x = P * X
    # Assume depth Z=1 for simplicity
    x_h = np.array([x[0], x[1], 1])  # Make sure x is in homogeneous form
    M = P[:, :3]  # Extract the camera intrinsic matrix
    d = np.linalg.inv(M) @ x_h  # Backproject the point
    d = d / np.linalg.norm(d)  # Normalize the direction vector
    return d



In [48]:
def intersection_line_of_planes(n1, d1, n2, d2):
    """
    Find the intersection line of two planes in 3D.

    Parameters:
        n1 (numpy.ndarray): Normal vector of plane 1.
        d1 (float): Plane 1 constant.
        n2 (numpy.ndarray): Normal vector of plane 2.
        d2 (float): Plane 2 constant.

    Returns:
        X0 (numpy.ndarray): A point on the intersection line.
        d (numpy.ndarray): Direction vector of the intersection line.
    """
    # Step 1: Direction of the intersection line (cross product of normals)
    d =  np.cross(n1, n2)
    
    # Check if the planes are parallel
    if np.linalg.norm(d) == 0:
        raise ValueError("Planes are parallel, no intersection line.")
    
    # Step 2: Find a point on the intersection line
    # Solve for a point X0 satisfying both plane equations
    A = np.vstack((n1, n2, d))  # Stack normal vectors and direction vector
    b = np.array([d1, d2, 0])   # Right-hand side with 0 for d direction
    
    # Solve the linear system for X0
    X0 = - np.linalg.lstsq(A, b, rcond=None)[0]
    
    return X0, d


In [49]:
import sys
import os
import matplotlib.pyplot as plt
sys.path.append(os.path.abspath('../utils'))
sys.path.append(os.path.abspath('../'))
import model2 as model2_module
import image as image_module
print(os.getcwd())
os.chdir('../')

model2_module.load_model()

/Users/kangyufeng/Documents/Repo
load gaze estimator
the pre-trained gaze estimation model does not exist.


FileNotFoundError: [Errno 2] No such file or directory: 'ckpt/epoch_24_ckpt.pth.tar'

In [50]:
image_path = "stereoCalibrate/data/7_left.JPG"  
img, small = image_module.load_img(image_path)



image_module.set_img(small)

print(img.shape)
# gaze estimation

arrow_start_left, arrow_end_left = model2_module.find_vector_arrow(img)


[ WARN:0@2123.282] global loadsave.cpp:248 findDecoder imread_('stereoCalibrate/data/7_left.JPG'): can't open/read file: check file path/integrity


error: OpenCV(4.9.0) /Users/xperience/GHA-OpenCV-Python2/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:196: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [26]:
plt.figure()
plt.imshow(img)
plt.scatter(arrow_start_left[0], arrow_start_left[1], c='r')
plt.show()



In [27]:
image_path = "stereoCalibrate/data/7_right.JPG"  
img, small = image_module.load_img(image_path)
# image_module.set_img(small)

arrow_start_right, arrow_end_right = model2_module.find_vector_arrow(img)

detected_faces:  3
pred_gaze_np:  [-0.48817384  0.65958476]


In [28]:
plt.figure()
plt.imshow(img)
plt.scatter(arrow_start_right[0], arrow_start_right[1], c='r')
plt.show()



In [29]:
face_3d = DLT(P1, P2, arrow_start_left, arrow_start_right)



In [51]:
v_left_1 = backproject(P1, arrow_start_left)
v_left_2 = backproject(P1, arrow_end_left)

v_right_1 = backproject(P2, arrow_start_right)
v_right_2 = backproject(P2, arrow_end_right)

n_left, d_left = find_plane_two_vectors(face_3d, v_left_1, v_left_2)
n_right, d_right = find_plane_two_vectors(c2, v_right_1, v_right_2)

x, d_line = intersection_line_of_planes(n_left, d_left, n_right, d_right)





In [52]:
fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')

pts_x_c = pts_x + [c1[0], c2[0]]
pts_y_c = pts_y + [c1[1], c2[1]]
pts_z_c = pts_z + [c1[2], c2[2]]


max_range = max([max(pts_x_c)-min(pts_x_c), max(pts_y_c)-min(pts_y_c), max(pts_z_c)-min(pts_z_c)])
mid_x = (max(pts_x_c) + min(pts_x_c)) * 0.5
mid_y = (max(pts_y_c) + min(pts_y_c)) * 0.5
mid_z = (max(pts_z_c) + min(pts_z_c)) * 0.5
ax.view_init(elev=135, azim=0)
ax.set_xlim(mid_x - max_range/2, mid_x + max_range/2)
ax.set_ylim(mid_y - max_range/2, mid_y + max_range/2)
ax.set_zlim(mid_z - max_range/2, mid_z + max_range/2)

ax.scatter(pts_x, pts_y, pts_z, c='b', label='Chessboard')
ax.scatter(c1[0], c1[1], c1[2], c='r', label='Camera 1')
ax.scatter(c2[0], c2[1], c2[2], c='purple',  label='Camera 2')
ax.scatter(face_3d[0], face_3d[1], face_3d[2], c='black', label='Face')
ax.scatter(pt_table[0][0], pt_table[0][1], pt_table[0][2], c='g', label='Table')
ax.scatter(pt_table[1][0], pt_table[1][1], pt_table[1][2], c='g' )
ax.scatter(pt_table[2][0], pt_table[2][1], pt_table[2][2], c='g')
X_range = np.linspace(mid_x - max_range/2, mid_x + max_range/2, 10)
Y_range = np.linspace(mid_y - max_range/2, mid_y + max_range/2, 10)
n_x, n_y, n_z = normal
X, Y = np.meshgrid(X_range, Y_range)
Z = (-d - n_x * X - n_y * Y) / n_z
ax.plot_surface(X, Y, Z, alpha=0.5, color='cyan')
t = np.linspace(-200, 200, 100)

# X_line = c2[0] + t * v_right_1[0]
# Y_line = c2[1] + t * v_right_1[1]
# Z_line = c2[2] + t * v_right_1[2]
# ax.plot(X_line, Y_line, Z_line, c='r')
# ax.plot_surface(X, Y, Z, alpha=0.5, color='cyan')

# X = c2[0] + t * v_right_2[0]    
# Y = c2[1] + t * v_right_2[1]
# Z = c2[2] + t * v_right_2[2]
# ax.plot(X, Y, Z, c='r')
# Z2 = (-d_left - n_left[0] * X - n_left[1] * Y) / n_left[2]
# Z3 = (-d_right - n_right[0] * X - n_right[1] * Y) / n_right[2]

# ax.plot_surface(X, Y, Z2, alpha=0.5, color='yellow')
# ax.plot_surface(X, Y, Z3, alpha=0.5, color='red')

X = x[0] + t * d_line[0]
Y = x[1] + t * d_line[1]
Z = x[2] + t * d_line[2]


ax.plot(X, Y, Z, c='orange', label='Gaze vector')
ax.legend()
plt.show()

In [ ]:
import sys
import os

sys.path.append(os.path.abspath('../utils'))
sys.path.append(os.path.abspath('../'))


print(os.chdir("../"))
import model1 as model_module
model_module.time_show()

model_module.load_model("vit_b", "./vit_b.pth")

None
Time will be shown
Loading model
Model loaded
Time taken to load model:  0.378673791885376


In [ ]:
small = cv.resize(im_left, (224, 224))

mask = model_module.mask_generate(small)



print(mask)

Generating mask
Time taken to generate mask:  31.678187131881714
[{'segmentation': array([[ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]]), 'area': 228, 'bbox': [0, 0, 17, 12], 'predicted_iou': 1.008114218711853, 'point_coords': [[10.5, 10.5]], 'stability_score': 0.9912280440330505, 'crop_box': [0, 0, 224, 224]}, {'segmentation': array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]]), 'area': 435, 'bbox': [106

In [11]:
import image as image_module

image_module.set_img(small)

image_module.show_anns(mask)

2024-12-17 20:40:26.669 python[33004:8346797] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-17 20:40:26.669 python[33004:8346797] +[IMKInputSession subclass]: chose IMKInputSession_Modern
